In [1]:
import pandas as pd

In [2]:
fb=pd.read_csv("C:\\ksr\\data science\\DS_batch1\\datasets\\Fiberbits.csv")

In [3]:
fb.shape


(100000, 9)

In [4]:

# import packages# import 
import pandas as pd
import numpy as np
import pandas.core.algorithms as algos
from pandas import Series
import scipy.stats.stats as stats
import re
import traceback
import string

max_bin = 20
force_bin = 5

# define a binning function
def mono_bin(Y, X, n = max_bin):
    
    df1 = pd.DataFrame({"X": X, "Y": Y})
    justmiss = df1[['X','Y']][df1.X.isnull()]
    notmiss = df1[['X','Y']][df1.X.notnull()]
    r = 0
    while np.abs(r) < 1:
        try:
            d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.qcut(notmiss.X, n)})
            d2 = d1.groupby('Bucket', as_index=True)
            r, p = stats.spearmanr(d2.mean().X, d2.mean().Y)
            n = n - 1 
        except Exception as e:
            n = n - 1

    if len(d2) == 1:
        n = force_bin         
        bins = algos.quantile(notmiss.X, np.linspace(0, 1, n))
        if len(np.unique(bins)) == 2:
            bins = np.insert(bins, 0, 1)
            bins[1] = bins[1]-(bins[1]/2)
        d1 = pd.DataFrame({"X": notmiss.X, "Y": notmiss.Y, "Bucket": pd.cut(notmiss.X, np.unique(bins),include_lowest=True)}) 
        d2 = d1.groupby('Bucket', as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["MIN_VALUE"] = d2.min().X
    d3["MAX_VALUE"] = d2.max().X
    d3["COUNT"] = d2.count().Y
    d3["EVENT"] = d2.sum().Y
    d3["NONEVENT"] = d2.count().Y - d2.sum().Y
    d3=d3.reset_index(drop=True)
    
    if len(justmiss.index) > 0:
        d4 = pd.DataFrame({'MIN_VALUE':np.nan},index=[0])
        d4["MAX_VALUE"] = np.nan
        d4["COUNT"] = justmiss.count().Y
        d4["EVENT"] = justmiss.sum().Y
        d4["NONEVENT"] = justmiss.count().Y - justmiss.sum().Y
        d3 = d3.append(d4,ignore_index=True)
    
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    
    return(d3)

def char_bin(Y, X):
        
    df1 = pd.DataFrame({"X": X, "Y": Y})
    df2 = df1.groupby('X',as_index=True)
    
    d3 = pd.DataFrame({},index=[])
    d3["COUNT"] = df2.count().Y
    d3["MIN_VALUE"] = df2.groups
    d3["MAX_VALUE"] = df2.groups
    d3["EVENT"] = df2.sum().Y
    d3["NONEVENT"] = df2.count().Y - df2.sum().Y     
    d3["EVENT_RATE"] = d3.EVENT/d3.sum().EVENT
    d3["NON_EVENT_RATE"] = d3.NONEVENT/d3.sum().NONEVENT
    d3["WOE"] = np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["IV"] = (d3.EVENT_RATE-d3.NON_EVENT_RATE)*np.log(d3.EVENT_RATE/d3.NON_EVENT_RATE)
    d3["VAR_NAME"] = "VAR"
    d3 = d3[['VAR_NAME','MIN_VALUE', 'MAX_VALUE', 'COUNT', 'EVENT', 'EVENT_RATE', 'NONEVENT', 'NON_EVENT_RATE', 'WOE', 'IV']]       
    d3 = d3.replace([np.inf, -np.inf], 0)
    d3.IV = d3.IV.sum()
    d3 = d3.reset_index(drop=True)
    
    return(d3)

def data_vars(df1, target):
    
    stack = traceback.extract_stack()
    filename, lineno, function_name, code = stack[-2]
    vars_name = re.compile(r'\((.*?)\).*$').search(code).groups()[0]
    final = (re.findall(r"[\w']+", vars_name))[-1]
    
    x = df1.dtypes.index
    count = -1
    
    for i in x:
        if i.upper() not in (final.upper()):
            if np.issubdtype(df1[i], np.number) and len(Series.unique(df1[i])) > 2:
                conv = mono_bin(target, df1[i])
                conv["VAR_NAME"] = i
                count = count + 1
            else:
                conv = char_bin(target, df1[i])
                conv["VAR_NAME"] = i            
                count = count + 1
                
            if count == 0:
                iv_df = conv
            else:
                iv_df = iv_df.append(conv,ignore_index=True)
    
    iv = pd.DataFrame({'IV':iv_df.groupby('VAR_NAME').IV.max()})
    iv = iv.reset_index()
    return(iv_df,iv)






In [5]:
IV = data_vars(fb,fb["active_cust"])
IV

(                      VAR_NAME  MIN_VALUE  MAX_VALUE  COUNT  EVENT  \
 0                       income       1130       1674  25202  20108   
 1                       income       1675       1805  24951  14705   
 2                       income       1806       1907  24936   9249   
 3                       income       1908     229821  24911  13797   
 4            months_on_network        -20         50  35426  17064   
 5            months_on_network         51         66  32874  16786   
 6            months_on_network         67        449  31700  24009   
 7               Num_complaints          3          4  95415  55984   
 8               Num_complaints          5          6   4585   1875   
 9          number_plan_changes          0          1  61372  41845   
 10         number_plan_changes          2          2  25757  13742   
 11         number_plan_changes          3          9  12871   2272   
 12                   relocated          0          0  87652  56905   
 13   

In [6]:
IV[1]

VAR_NAME        IV
0              Num_complaints  0.022933
1           Speed_test_result  0.508128
2                      income  0.421936
3                monthly_bill  0.014496
4           months_on_network  0.271047
5         number_plan_changes  0.518536
6                   relocated  0.785984
7  technical_issues_per_month  0.346512

In [7]:
IV1=IV[1]

In [8]:
IV1

VAR_NAME        IV
0              Num_complaints  0.022933
1           Speed_test_result  0.508128
2                      income  0.421936
3                monthly_bill  0.014496
4           months_on_network  0.271047
5         number_plan_changes  0.518536
6                   relocated  0.785984
7  technical_issues_per_month  0.346512

In [11]:
IV1["flag"]=np.where((IV1["IV"]>0.02)&(IV1["IV"]<=0.5),1,0)

In [13]:
IV2=IV1[IV1["flag"]==1]

In [14]:
IV2

VAR_NAME        IV  flag
0              Num_complaints  0.022933     1
2                      income  0.421936     1
4           months_on_network  0.271047     1
7  technical_issues_per_month  0.346512     1

In [24]:
fb1=fb[IV2["VAR_NAME"]]

In [25]:
fb1.head()

Num_complaints  income  months_on_network  technical_issues_per_month
0               4    1586                 85                           4
1               4    1581                 85                           4
2               4    1594                 82                           4
3               4    1594                 82                           4
4               4    1609                 80                           4

In [26]:
fb1["active_cust"]=fb["active_cust"]

C:\Users\Koti\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
fb1.head()

Num_complaints  income  months_on_network  technical_issues_per_month  \
0               4    1586                 85                           4   
1               4    1581                 85                           4   
2               4    1594                 82                           4   
3               4    1594                 82                           4   
4               4    1609                 80                           4   

   active_cust  
0            0  
1            0  
2            0  
3            0  
4            1

In [27]:
x_train=fb1.drop(["active_cust"],1)
y_train=fb1["active_cust"]

In [28]:
from sklearn.linear_model import LogisticRegression
lg=LogisticRegression()
lg.fit(x_train,y_train)

C:\Users\Koti\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [29]:
lg.score(x_train,y_train)

0.67574

In [30]:
lg.classes_

array([0, 1], dtype=int64)

In [31]:
lg.coef_

array([[-0.69015098,  0.00141268,  0.04173874, -0.50065557]])

In [33]:
list(zip(x_train.columns,lg.coef_[0]))

[('Num_complaints', -0.69015098215682),
 ('income', 0.0014126780186493521),
 ('months_on_network', 0.041738743538266294),
 ('technical_issues_per_month', -0.5006555748639557)]

In [35]:
import statsmodels.formula.api as sm
model=sm.logit(formula="active_cust~Num_complaints+income+months_on_network+technical_issues_per_month",data=fb1).fit()
model.summary2()

Optimization terminated successfully.
         Current function value: 0.615826
         Iterations 9


<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Results: Logit
===========================================================================
Model:                 Logit               Pseudo R-squared:    0.095      
Dependent Variable:    active_cust         AIC:                 123175.1694
Date:                  2019-09-08 09:41    BIC:                 123222.7341
No. Observations:      100000              Log-Likelihood:      -61583.    
Df Model:              4                   LL-Null:             -68074.    
Df Residuals:          99995               LLR p-value:         0.0000     
Converged:             1.0000              Scale:               1.0000     
No. Iterations:        9.0000                                              
---------------------------------------------------------------------------
                            Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------------------------------
Intercept                  -3.2448   0.2545 -12.7511 0.0000 -3.7436 -2.7461
Num_complaints             -0.4803   0.0249 -19.3234 0.0000 -0.5290 -0.4316
income                      0.0023   0.0001  24.8420 0.0000  0.0021  0.0025
months_on_network           0.0525   0.0011  46.5240 0.0000  0.0503  0.0547
technical_issues_per_month -0.4913   0.0063 -77.4416 0.0000 -0.5037 -0.4789
===========================================================================

"""

In [36]:
import statsmodels.formula.api as sm
model=sm.logit(formula="active_cust~income+months_on_network+technical_issues_per_month",data=fb1).fit()
model.summary2()

Optimization terminated successfully.
         Current function value: 0.617748
         Iterations 9


<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Results: Logit
===========================================================================
Model:                 Logit               Pseudo R-squared:    0.093      
Dependent Variable:    active_cust         AIC:                 123557.6387
Date:                  2019-09-08 09:54    BIC:                 123595.6904
No. Observations:      100000              Log-Likelihood:      -61775.    
Df Model:              3                   LL-Null:             -68074.    
Df Residuals:          99996               LLR p-value:         0.0000     
Converged:             1.0000              Scale:               1.0000     
No. Iterations:        9.0000                                              
---------------------------------------------------------------------------
                            Coef.  Std.Err.    z     P>|z|   [0.025  0.975]
---------------------------------------------------------------------------
Intercept                  -5.4473   0.2348 -23.2040 0.0000 -5.9074 -4.9872
income                      0.0024   0.0001  25.2214 0.0000  0.0022  0.0026
months_on_network           0.0541   0.0011  47.0953 0.0000  0.0519  0.0564
technical_issues_per_month -0.4916   0.0063 -77.6353 0.0000 -0.5040 -0.4792
===========================================================================

"""